In [1]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 15.3 MB/s eta 0:00:00


In [2]:
!playwright install

172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 3.6s172.5 MiB [] 1% 2.4s172.5 MiB [] 2% 2.3s172.5 MiB [] 3% 2.0s172.5 MiB [] 4% 1.8s172.5 MiB [] 5% 1.8s172.5 MiB [] 5% 2.0s172.5 MiB [] 6% 1.9s172.5 MiB [] 7% 2.0s172.5 MiB [] 8% 1.8s172.5 MiB [] 9% 1.8s172.5 MiB [] 10% 1.8s172.5 MiB [] 12% 1.7s172.5 MiB [] 13% 1.7s172.5 MiB [] 14% 1.6s172.5 MiB [] 16% 1.6s172.5 MiB [] 17% 1.5s172.5 MiB [] 18% 1.4s172.5 MiB [] 20% 1.4s172.5 MiB [] 21% 1.4s172.5 MiB [] 22% 1.3s172.5 MiB [] 23% 1.3s172.5 MiB [] 24% 1.3s172.5 MiB [] 25% 1.2s172.5 MiB [] 26% 1.3s172.5 MiB [] 28% 1.3s172.5 MiB [] 29% 1.2s172.5 MiB [] 30% 1.2s172.5 MiB [] 31% 1.2s172.5 MiB [] 32% 1.1s172.5 MiB [] 34% 1.1s172.5 MiB [] 35% 1.1s172.5 MiB [] 37% 1.0s172.5 MiB [] 38% 1.0s172.5 MiB [] 40% 0.9s172.5 MiB [] 41% 0.9s172.5 MiB [] 43% 0.9s172.5 MiB [] 44% 0.8s172.5 MiB [] 46% 0.8s172.5 MiB [] 47% 0.8s172.5 MiB [] 49% 0.7s172.5 MiB [] 51% 0.7s172.5 MiB [] 52% 0.7s172.5 MiB [] 54% 0.7s172.5 MiB [] 55% 0.6s172.5 MiB [] 56% 0.6s172.5 MiB [] 58% 0.6s172.

In [3]:
!pip install openai

In [27]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [29]:
#to test if the API key is working

openai = OpenAI()
prompt = "hello, who are you?"
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ {"role": "user", "content": prompt}]
)
response.choices[0].message.content

"Hello! I'm an AI language model created to assist you with a variety of questions and tasks. How can I help you today?"

In [30]:
!pip install gradio

In [31]:
import gradio as gr

In [52]:
import os
from google.colab import userdata
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [53]:
!pip install requests

In [58]:
import requests
import os

#Serper API key and the search query
api_key = os.environ.get("SERPER_API_KEY")
query = "latest developments in generative AI"

#URL
url = "https://google.serper.dev/search"


headers = {
    'X-API-KEY': api_key,
    'Content-Type': 'application/json'
}


payload = {
    "q": query,
    "num": 3  # Number of search results
}

# Send a POST request to Serper API
response = requests.post(url, headers=headers, json=payload)

# Check request
response.raise_for_status()


data = response.json()


search_results = []
if 'organic' in data:
    for item in data['organic'][:3]: # Take top 3 organic results
        search_results.append({
            'title': item.get('title'),
            'url': item.get('link')
        })




print("Top search results:")
if search_results:
    for result in search_results:
        print(f"- {result.get('title')} ({result.get('url')})")
else:
    print("No organic search results found.")

Top search results:
- Generative AI news and analysis - TechCrunch (https://techcrunch.com/tag/generative-ai/)
- Latest AI Breakthroughs and News: May, June, July 2025 (https://www.crescendo.ai/news/latest-ai-news-and-updates)
- 5 Generative AI Trends To Watch Out For In 2025 - Forbes (https://www.forbes.com/sites/janakirammsv/2025/01/12/5-generative-ai-trends-to-watch-out-for-in-2025/)


In [55]:
import asyncio
from playwright.async_api import async_playwright
import openai
from openai import OpenAI
import getpass

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

async def search_and_summarize(urls):
    async with async_playwright() as p:
        # Launch a Chromium browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        all_text_content = ""
        print("Scraping content from provided URLs...")
        for url_info in urls:
            url = url_info.get('url')
            if url:
                try:
                    await page.goto(url, timeout=30000)
                    content_elements = await page.locator('body p').all()

                    for element in content_elements:
                        text = await element.inner_text()
                        all_text_content += text + "\n"
                    print(f"Successfully scraped: {url}")
                except Exception as e:
                    print(f"Could not scrape {url}: {e}")

        if not all_text_content:
            await browser.close()
            return "Could not find sufficient content to summarize from the provided URLs."

        #Using OpenAI to summarize the content
        print("\nSummarizing content...")

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes web page content."},
                {"role": "user", "content": f"Based on the following text from several web pages, provide a concise summary of the key findings in no more than 250 words:\n\n{all_text_content}"}
            ]
        )

        summary = completion.choices[0].message.content
        await browser.close()
        return summary


In [59]:
# call the async function
final_summary = await search_and_summarize(search_results)

print("\nFinal Summary:")
print(final_summary)

Scraping content from provided URLs...
Successfully scraped: https://techcrunch.com/tag/generative-ai/
Successfully scraped: https://www.crescendo.ai/news/latest-ai-news-and-updates
Successfully scraped: https://www.forbes.com/sites/janakirammsv/2025/01/12/5-generative-ai-trends-to-watch-out-for-in-2025/

Summarizing content...

Final Summary:
The AI landscape is rapidly evolving, with significant developments across various sectors. Key findings include:

1. **Generative AI Trends**: The trend is shifting from integrating AI as a feature to embracing an AI-first approach in application development. This includes advances in coding assistants and AI agents that automate customer interactions and decision-making, marking a significant transformation in software as a service (SaaS).

2. **Government Initiatives**: There's increasing focus on AI regulation, with bipartisan task forces, state legislation, and international collaborations aiming to establish frameworks for ethical AI use. A

In [60]:
import gradio as gr
import requests
import os
import asyncio

async def get_urls_from_serper(query):
    """Fetches search results URLs from Serper API."""
    api_key = os.environ.get("SERPER_API_KEY")
    url = "https://google.serper.dev/search"
    headers = {
        'X-API-KEY': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        "q": query,
        "num": 3 # Number of search results
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        search_results = []
        if 'organic' in data:
            for item in data['organic'][:3]:
                search_results.append({'title': item.get('title'), 'url': item.get('link')})
        return search_results
    except Exception as e:
        print(f"Error fetching search results from Serper API: {e}")
        return []

async def summarize_search_results(query):
    """Performs search, scrapes content, and summarizes using OpenAI."""
    urls = await get_urls_from_serper(query)
    if not urls:
        return "Could not retrieve search results from Serper API."

    summary = await search_and_summarize(urls)
    return summary


#Gradio interface
iface = gr.Interface(
    fn=lambda query: asyncio.run(summarize_search_results(query)),
    inputs=gr.Textbox(label="Enter your search query"),
    outputs=gr.Textbox(label="Summary"),
    title="Serper API Search and Summarization",
    description="Enter a search query to get a summary of the top search results using Serper API and OpenAI."
)


iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ace24c517bf58184a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
